In [1]:
from timeit import default_timer as timer
import pandas as pd
import scipy.stats
import numpy as np
import datetime as dt
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
start_time = timer()

In [2]:
data = pd.read_csv('applications data.csv')

In [3]:
data.shape

(1000000, 10)

In [4]:
data.head(10)

,record,date,ssn,firstname,lastname,address,zip5,dob,homephone,fraud_label
0,1,20160101,379070012,XRRAMMTR,SMJETJMJ,6861 EUTST PL,2765,19070626,1797504115,0
1,2,20160101,387482503,MAMSTUJR,RTTEMRRR,7280 URASA PL,57169,19340615,4164239415,1
2,3,20160101,200332444,SZMMUJEZS,EUSEZRAE,5581 RSREX LN,56721,19070626,216537580,0
3,4,20160101,747451317,SJJZSXRSZ,ETJXTXXS,1387 UJZXJ RD,35286,19440430,132144161,0
4,5,20160101,24065868,SSSXUEJMS,SSUUJXUZ,279 EAASA WY,3173,19980315,6101082272,0
5,6,20160101,922264214,XEEJJSTER,ERJSAXA,4322 USJXU LN,8391,19480613,5742016319,0
6,7,20160101,415812149,XZJRJUSRR,STSMJRUM,478 EEXUM LN,41640,19640318,7697396312,0
7,8,20160101,373752050,EJMRRSUXR,AMTZXRU,8906 UUAJ PL,60567,19190528,2727421586,0
8,9,20160101,769970791,RXTSZJATS,RSXMRJME,8266 SSEAR RD,37934,19900314,5520652307,0
9,10,20160101,732119085,EREMTZXXA,EXMMSEXT,9166 XXSAZ PL,93751,19750127,3749451123,0


In [5]:
data.dtypes

record          int64
date            int64
ssn             int64
firstname      object
lastname       object
address        object
zip5            int64
dob             int64
homephone       int64
fraud_label     int64
dtype: object

In [6]:
data['appyear'] = (data['date'] / 10000).astype('int')
data['appmonth'] = ((data['date'] - data['appyear'] * 10000) / 100).astype('int')
data['appday'] = data['date'] - data['appyear'] * 10000 - data['appmonth'] * 100
data['date'] = pd.to_datetime(data.appyear*10000+data.appmonth*100+data.appday,format='%Y%m%d')
data = data.drop(columns = ['appyear','appmonth','appday'])

In [7]:
data.dtypes

record                  int64
date           datetime64[ns]
ssn                     int64
firstname              object
lastname               object
address                object
zip5                    int64
dob                     int64
homephone               int64
fraud_label             int64
dtype: object

In [8]:
data.head(5)

,record,date,ssn,firstname,lastname,address,zip5,dob,homephone,fraud_label
0,1,2016-01-01,379070012,XRRAMMTR,SMJETJMJ,6861 EUTST PL,2765,19070626,1797504115,0
1,2,2016-01-01,387482503,MAMSTUJR,RTTEMRRR,7280 URASA PL,57169,19340615,4164239415,1
2,3,2016-01-01,200332444,SZMMUJEZS,EUSEZRAE,5581 RSREX LN,56721,19070626,216537580,0
3,4,2016-01-01,747451317,SJJZSXRSZ,ETJXTXXS,1387 UJZXJ RD,35286,19440430,132144161,0
4,5,2016-01-01,24065868,SSSXUEJMS,SSUUJXUZ,279 EAASA WY,3173,19980315,6101082272,0


In [9]:
data.describe()

,record,ssn,zip5,dob,homephone,fraud_label
count,1000000.000000,1.000000e+06,1000000.000000,1.000000e+06,1.000000e+06,1000000.000000
mean,500000.500000,5.083415e+08,50073.333143,1.951725e+07,5.368716e+09,0.014393
std,288675.278933,2.934864e+08,28915.921546,3.568872e+05,3.082161e+09,0.119104
min,1.000000,3.600000e+01,2.000000,1.900010e+07,5.937990e+05,0.000000
25%,250000.750000,2.542176e+08,24992.000000,1.916092e+07,2.677134e+09,0.000000
50%,500000.500000,5.088144e+08,50492.000000,1.950090e+07,5.399910e+09,0.000000
75%,750000.250000,7.630924e+08,74496.000000,1.983032e+07,8.109241e+09,0.000000
max,1000000.000000,1.000000e+09,99999.000000,2.016103e+07,1.000000e+10,1.000000


In [10]:
numrecords = len(data)
print(numrecords)

1000000


## Assignment 6

In [11]:
data

,record,date,ssn,firstname,lastname,address,zip5,dob,homephone,fraud_label
0,1,2016-01-01,379070012,XRRAMMTR,SMJETJMJ,6861 EUTST PL,2765,19070626,1797504115,0
1,2,2016-01-01,387482503,MAMSTUJR,RTTEMRRR,7280 URASA PL,57169,19340615,4164239415,1
2,3,2016-01-01,200332444,SZMMUJEZS,EUSEZRAE,5581 RSREX LN,56721,19070626,216537580,0
3,4,2016-01-01,747451317,SJJZSXRSZ,ETJXTXXS,1387 UJZXJ RD,35286,19440430,132144161,0
4,5,2016-01-01,24065868,SSSXUEJMS,SSUUJXUZ,279 EAASA WY,3173,19980315,6101082272,0
...,...,...,...,...,...,...,...,...,...,...
999995,999996,2016-12-31,766104848,XUSAAURMZ,EJXXEXZX,8079 EJRUJ AVE,54890,19550418,8236558223,0
999996,999997,2016-12-31,445178058,SAAJJEMTX,UMAUETJX,4817 UMSXT AVE,74683,19150624,6790134432,0
999997,999998,2016-12-31,25283410,ESRXXRSUX,EUEJRJJE,7562 RJMAZ ST,86805,19880628,3997657622,0
999998,999999,2016-12-31,361232148,XXMZSRXMA,RSRSUJSS,3408 UZZAM WY,49183,20120302,5935293731,0


In [12]:
data.dtypes

record                  int64
date           datetime64[ns]
ssn                     int64
firstname              object
lastname               object
address                object
zip5                    int64
dob                     int64
homephone               int64
fraud_label             int64
dtype: object

In [13]:
import datetime as dt
from datetime import datetime

In [14]:
data['zip5'] = data['zip5'].apply(lambda x: '{0:0>5}'.format(x))
data

,record,date,ssn,firstname,lastname,address,zip5,dob,homephone,fraud_label
0,1,2016-01-01,379070012,XRRAMMTR,SMJETJMJ,6861 EUTST PL,02765,19070626,1797504115,0
1,2,2016-01-01,387482503,MAMSTUJR,RTTEMRRR,7280 URASA PL,57169,19340615,4164239415,1
2,3,2016-01-01,200332444,SZMMUJEZS,EUSEZRAE,5581 RSREX LN,56721,19070626,216537580,0
3,4,2016-01-01,747451317,SJJZSXRSZ,ETJXTXXS,1387 UJZXJ RD,35286,19440430,132144161,0
4,5,2016-01-01,24065868,SSSXUEJMS,SSUUJXUZ,279 EAASA WY,03173,19980315,6101082272,0
...,...,...,...,...,...,...,...,...,...,...
999995,999996,2016-12-31,766104848,XUSAAURMZ,EJXXEXZX,8079 EJRUJ AVE,54890,19550418,8236558223,0
999996,999997,2016-12-31,445178058,SAAJJEMTX,UMAUETJX,4817 UMSXT AVE,74683,19150624,6790134432,0
999997,999998,2016-12-31,25283410,ESRXXRSUX,EUEJRJJE,7562 RJMAZ ST,86805,19880628,3997657622,0
999998,999999,2016-12-31,361232148,XXMZSRXMA,RSRSUJSS,3408 UZZAM WY,49183,20120302,5935293731,0


In [15]:
data.loc[data.ssn == 999999999, 'ssn'] = -data[data.ssn == 999999999]['record']
data['ssn'] = data['ssn'].apply(lambda x: '{0:0>9}'.format(x))
data

,record,date,ssn,firstname,lastname,address,zip5,dob,homephone,fraud_label
0,1,2016-01-01,379070012,XRRAMMTR,SMJETJMJ,6861 EUTST PL,02765,19070626,1797504115,0
1,2,2016-01-01,387482503,MAMSTUJR,RTTEMRRR,7280 URASA PL,57169,19340615,4164239415,1
2,3,2016-01-01,200332444,SZMMUJEZS,EUSEZRAE,5581 RSREX LN,56721,19070626,216537580,0
3,4,2016-01-01,747451317,SJJZSXRSZ,ETJXTXXS,1387 UJZXJ RD,35286,19440430,132144161,0
4,5,2016-01-01,024065868,SSSXUEJMS,SSUUJXUZ,279 EAASA WY,03173,19980315,6101082272,0
...,...,...,...,...,...,...,...,...,...,...
999995,999996,2016-12-31,766104848,XUSAAURMZ,EJXXEXZX,8079 EJRUJ AVE,54890,19550418,8236558223,0
999996,999997,2016-12-31,445178058,SAAJJEMTX,UMAUETJX,4817 UMSXT AVE,74683,19150624,6790134432,0
999997,999998,2016-12-31,025283410,ESRXXRSUX,EUEJRJJE,7562 RJMAZ ST,86805,19880628,3997657622,0
999998,999999,2016-12-31,361232148,XXMZSRXMA,RSRSUJSS,3408 UZZAM WY,49183,20120302,5935293731,0


In [16]:
data['address'] = np.where(data['address'] == "123 MAIN ST", -data['record'], data['address'])
data['address'] = data['address'].astype(str)

In [17]:
data.head(100)

,record,date,ssn,firstname,lastname,address,zip5,dob,homephone,fraud_label
0,1,2016-01-01,379070012,XRRAMMTR,SMJETJMJ,6861 EUTST PL,02765,19070626,1797504115,0
1,2,2016-01-01,387482503,MAMSTUJR,RTTEMRRR,7280 URASA PL,57169,19340615,4164239415,1
2,3,2016-01-01,200332444,SZMMUJEZS,EUSEZRAE,5581 RSREX LN,56721,19070626,216537580,0
3,4,2016-01-01,747451317,SJJZSXRSZ,ETJXTXXS,1387 UJZXJ RD,35286,19440430,132144161,0
4,5,2016-01-01,024065868,SSSXUEJMS,SSUUJXUZ,279 EAASA WY,03173,19980315,6101082272,0
...,...,...,...,...,...,...,...,...,...,...
95,96,2016-01-01,939863686,UUZMJST,RJURSTXJ,7277 XEART CT,69109,19921029,7393643883,0
96,97,2016-01-01,960805076,XRTAEZASZ,SUERZXAT,8411 URMMX RD,20001,19950822,1458629808,0
97,98,2016-01-01,695355308,ZMEAJXJU,AXRTEJE,5743 RTRTX ST,25254,19200526,5421614404,0
98,99,2016-01-01,870939019,ETREZZUXS,EAZSEEJE,200 UUUJM AVE,05507,19770415,9419642060,0


In [18]:
data.loc[data.dob == 19070626, 'dob'] = -data.loc[data.dob == 19070626]['record']
data['dob'] = data['dob'].apply(lambda x:'{0:0>8}'.format(x))
data

,record,date,ssn,firstname,lastname,address,zip5,dob,homephone,fraud_label
0,1,2016-01-01,379070012,XRRAMMTR,SMJETJMJ,6861 EUTST PL,02765,000000-1,1797504115,0
1,2,2016-01-01,387482503,MAMSTUJR,RTTEMRRR,7280 URASA PL,57169,19340615,4164239415,1
2,3,2016-01-01,200332444,SZMMUJEZS,EUSEZRAE,5581 RSREX LN,56721,000000-3,216537580,0
3,4,2016-01-01,747451317,SJJZSXRSZ,ETJXTXXS,1387 UJZXJ RD,35286,19440430,132144161,0
4,5,2016-01-01,024065868,SSSXUEJMS,SSUUJXUZ,279 EAASA WY,03173,19980315,6101082272,0
...,...,...,...,...,...,...,...,...,...,...
999995,999996,2016-12-31,766104848,XUSAAURMZ,EJXXEXZX,8079 EJRUJ AVE,54890,19550418,8236558223,0
999996,999997,2016-12-31,445178058,SAAJJEMTX,UMAUETJX,4817 UMSXT AVE,74683,19150624,6790134432,0
999997,999998,2016-12-31,025283410,ESRXXRSUX,EUEJRJJE,7562 RJMAZ ST,86805,19880628,3997657622,0
999998,999999,2016-12-31,361232148,XXMZSRXMA,RSRSUJSS,3408 UZZAM WY,49183,20120302,5935293731,0


In [19]:
data.loc[data.homephone == 9999999999, 'homephone'] = data.loc[data.homephone == 9999999999]['record']
data['homephone'] = data['homephone'].apply(lambda x: '{0:0>10}'.format(x))
data

,record,date,ssn,firstname,lastname,address,zip5,dob,homephone,fraud_label
0,1,2016-01-01,379070012,XRRAMMTR,SMJETJMJ,6861 EUTST PL,02765,000000-1,1797504115,0
1,2,2016-01-01,387482503,MAMSTUJR,RTTEMRRR,7280 URASA PL,57169,19340615,4164239415,1
2,3,2016-01-01,200332444,SZMMUJEZS,EUSEZRAE,5581 RSREX LN,56721,000000-3,0216537580,0
3,4,2016-01-01,747451317,SJJZSXRSZ,ETJXTXXS,1387 UJZXJ RD,35286,19440430,0132144161,0
4,5,2016-01-01,024065868,SSSXUEJMS,SSUUJXUZ,279 EAASA WY,03173,19980315,6101082272,0
...,...,...,...,...,...,...,...,...,...,...
999995,999996,2016-12-31,766104848,XUSAAURMZ,EJXXEXZX,8079 EJRUJ AVE,54890,19550418,8236558223,0
999996,999997,2016-12-31,445178058,SAAJJEMTX,UMAUETJX,4817 UMSXT AVE,74683,19150624,6790134432,0
999997,999998,2016-12-31,025283410,ESRXXRSUX,EUEJRJJE,7562 RJMAZ ST,86805,19880628,3997657622,0
999998,999999,2016-12-31,361232148,XXMZSRXMA,RSRSUJSS,3408 UZZAM WY,49183,20120302,5935293731,0


In [20]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 10 columns):
 #   Column       Non-Null Count    Dtype         
---  ------       --------------    -----         
 0   record       1000000 non-null  int64         
 1   date         1000000 non-null  datetime64[ns]
 2   ssn          1000000 non-null  object        
 3   firstname    1000000 non-null  object        
 4   lastname     1000000 non-null  object        
 5   address      1000000 non-null  object        
 6   zip5         1000000 non-null  object        
 7   dob          1000000 non-null  object        
 8   homephone    1000000 non-null  object        
 9   fraud_label  1000000 non-null  int64         
dtypes: datetime64[ns](1), int64(2), object(7)
memory usage: 76.3+ MB


In [21]:
import calendar
data['dow'] = data.date.apply(lambda x: calendar.day_name[x.weekday()])

In [22]:
train_test = data[data.date < '2016-11-01']

In [23]:
c = 4; nmid = 20; y_avg = train_test['fraud_label'].mean()
y_dow = train_test.groupby('dow')['fraud_label'].mean()
num = train_test.groupby('dow').size()
y_dow_smooth = y_avg + (y_dow - y_avg)/(1 + np.exp(-(num-nmid)/c))
data['dow_risk'] = data.dow.map(y_dow_smooth)

In [24]:
y_dow = y_dow.reset_index()
cats = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
y_dow['dow'] = pd.Categorical(y_dow['dow'], categories = cats, ordered = True)
y_dow = y_dow.sort_values('dow')
y_dow = y_dow.set_index('dow')

In [25]:
#plt.style.use('ggplot')
#fig, ax = plt.subplots(figsize = (20,10))
#plt.bar(data = y_dow, x=y_dow.index, height = 'fraud_label', color = 'darkblue')
#ax.set_ylim(bottom = 0.013)
#ax.axhline(y = y_avg, ls = '--', lw = 2)

#for i, v in enumerate(y_dow.index):
#    ax.text(v, y_dow.loc[i, 'fraud_label'] + 0.0001, round(y_dow.loc[i, 'fraud_label'], 5), horizontalalignment = 'center', fontsize = 15)

#plt.show()

In [26]:
data

,record,date,ssn,firstname,lastname,address,zip5,dob,homephone,fraud_label,dow,dow_risk
0,1,2016-01-01,379070012,XRRAMMTR,SMJETJMJ,6861 EUTST PL,02765,000000-1,1797504115,0,Friday,0.014499
1,2,2016-01-01,387482503,MAMSTUJR,RTTEMRRR,7280 URASA PL,57169,19340615,4164239415,1,Friday,0.014499
2,3,2016-01-01,200332444,SZMMUJEZS,EUSEZRAE,5581 RSREX LN,56721,000000-3,0216537580,0,Friday,0.014499
3,4,2016-01-01,747451317,SJJZSXRSZ,ETJXTXXS,1387 UJZXJ RD,35286,19440430,0132144161,0,Friday,0.014499
4,5,2016-01-01,024065868,SSSXUEJMS,SSUUJXUZ,279 EAASA WY,03173,19980315,6101082272,0,Friday,0.014499
...,...,...,...,...,...,...,...,...,...,...,...,...
999995,999996,2016-12-31,766104848,XUSAAURMZ,EJXXEXZX,8079 EJRUJ AVE,54890,19550418,8236558223,0,Saturday,0.014968
999996,999997,2016-12-31,445178058,SAAJJEMTX,UMAUETJX,4817 UMSXT AVE,74683,19150624,6790134432,0,Saturday,0.014968
999997,999998,2016-12-31,025283410,ESRXXRSUX,EUEJRJJE,7562 RJMAZ ST,86805,19880628,3997657622,0,Saturday,0.014968
999998,999999,2016-12-31,361232148,XXMZSRXMA,RSRSUJSS,3408 UZZAM WY,49183,20120302,5935293731,0,Saturday,0.014968


In [27]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 12 columns):
 #   Column       Non-Null Count    Dtype         
---  ------       --------------    -----         
 0   record       1000000 non-null  int64         
 1   date         1000000 non-null  datetime64[ns]
 2   ssn          1000000 non-null  object        
 3   firstname    1000000 non-null  object        
 4   lastname     1000000 non-null  object        
 5   address      1000000 non-null  object        
 6   zip5         1000000 non-null  object        
 7   dob          1000000 non-null  object        
 8   homephone    1000000 non-null  object        
 9   fraud_label  1000000 non-null  int64         
 10  dow          1000000 non-null  object        
 11  dow_risk     1000000 non-null  float64       
dtypes: datetime64[ns](1), float64(1), int64(2), object(8)
memory usage: 91.6+ MB


In [28]:
data['name'] = data.firstname + data.lastname
data['fulladdress'] = data.address + data.zip5
data['name_dob'] = data.name + data.dob
data['name_fulladdress'] = data.name + data.fulladdress
data['name_homephone'] = data.name + data.homephone
data['fulladdress_dob'] = data.fulladdress + data.dob
data['fulladdress_homephone'] = data.fulladdress + data.homephone
data['dob_homephone'] = data.dob + data.homephone
data['homephone_name_dob'] = data.homephone + data.name_dob
data['fulladdress_name_dob'] = data.fulladdress + data.name_dob

In [29]:
data['name_zip5'] = data.name + data.zip5
data['name_address'] = data.name + data.address

data['firstname_dob'] = data.firstname + data.dob
data['firstname_address'] = data.firstname + data.address
data['firstname_zip5'] = data.firstname + data.zip5
data['firstname_homephone'] = data.firstname + data.homephone
data['firstname_fulladdress'] = data.firstname + data.fulladdress

data['lastname_dob'] = data.lastname + data.dob
data['lastname_address'] = data.lastname + data.address
data['lastname_zip5'] = data.lastname + data.zip5
data['lastname_homephone'] = data.lastname + data.homephone
data['lastname_fulladdress'] = data.lastname + data.fulladdress

In [30]:
for field in list(data.iloc[:,np.r_[3:9, 12:22]].columns):
    data['ssn_' + field] = data.ssn + data[field]

In [31]:
(data.columns)

Index(['record', 'date', 'ssn', 'firstname', 'lastname', 'address', 'zip5',
       'dob', 'homephone', 'fraud_label', 'dow', 'dow_risk', 'name',
       'fulladdress', 'name_dob', 'name_fulladdress', 'name_homephone',
       'fulladdress_dob', 'fulladdress_homephone', 'dob_homephone',
       'homephone_name_dob', 'fulladdress_name_dob', 'name_zip5',
       'name_address', 'firstname_dob', 'firstname_address', 'firstname_zip5',
       'firstname_homephone', 'firstname_fulladdress', 'lastname_dob',
       'lastname_address', 'lastname_zip5', 'lastname_homephone',
       'lastname_fulladdress', 'ssn_firstname', 'ssn_lastname', 'ssn_address',
       'ssn_zip5', 'ssn_dob', 'ssn_homephone', 'ssn_name', 'ssn_fulladdress',
       'ssn_name_dob', 'ssn_name_fulladdress', 'ssn_name_homephone',
       'ssn_fulladdress_dob', 'ssn_fulladdress_homephone', 'ssn_dob_homephone',
       'ssn_homephone_name_dob', 'ssn_fulladdress_name_dob'],
      dtype='object')

In [32]:
attributes = list(data.iloc[:, np.r_[2, 5, 7, 8, 12:50]].columns)

In [33]:
len(attributes)

42

In [34]:
data1 = data.copy()
vars = data.copy()
data1['check_date'] = data1.date
data1['check_record'] = data1.record
vars.shape

(1000000, 50)

In [35]:
import time
begin = time.time()
for entity in attributes:
    st = time.time()
    data_1 = data1[['record', 'date', entity]]
    data_r = data1[['check_record', 'check_date', entity]]
    temp = pd.merge(data_1, data_r, left_on = entity, right_on = entity)
    
    day_since_data = temp[temp.record > temp.check_record][['record', 'date', 'check_date']].groupby('record')[['date', 'check_date']].last()
    mapper = (day_since_data.date - day_since_data.check_date).dt.days
    vars[entity + '_day_since'] = vars.record.map(mapper)
    vars[entity + '_day_since'].fillna(365, inplace = True)
    print(f'\n{entity}_day_since ---> Done')
    
    for offset_t in [0, 1, 3, 7, 14, 30]:
        count_day_data = temp[(temp.check_date >= (temp.date - dt.timedelta(offset_t))) & (temp.record >= temp.check_record)]
        col_name = f'{entity}_count_{offset_t}'
        mapper2 = count_day_data.groupby('record')[entity].count()
        vars[col_name] = vars.record.map(mapper2)
        
        print(f'{entity}_count_{str(offset_t)} ---> Done')
    
    print(f'Run time for entity {entity} -------- {time.time()-st}s')
    
print(f'Total run time: {(time.time() - begin)/60}min')
     


ssn_day_since ---> Done
ssn_count_0 ---> Done
ssn_count_1 ---> Done
ssn_count_3 ---> Done
ssn_count_7 ---> Done
ssn_count_14 ---> Done
ssn_count_30 ---> Done
Run time for entity ssn -------- 2.8155629634857178s

address_day_since ---> Done
address_count_0 ---> Done
address_count_1 ---> Done
address_count_3 ---> Done
address_count_7 ---> Done
address_count_14 ---> Done
address_count_30 ---> Done
Run time for entity address -------- 2.8519701957702637s

dob_day_since ---> Done
dob_count_0 ---> Done
dob_count_1 ---> Done
dob_count_3 ---> Done
dob_count_7 ---> Done
dob_count_14 ---> Done
dob_count_30 ---> Done
Run time for entity dob -------- 19.2203369140625s

homephone_day_since ---> Done
homephone_count_0 ---> Done
homephone_count_1 ---> Done
homephone_count_3 ---> Done
homephone_count_7 ---> Done
homephone_count_14 ---> Done
homephone_count_30 ---> Done
Run time for entity homephone -------- 12.872170448303223s

name_day_since ---> Done
name_count_0 ---> Done
name_count_1 ---> Done
na


ssn_address_day_since ---> Done
ssn_address_count_0 ---> Done
ssn_address_count_1 ---> Done
ssn_address_count_3 ---> Done
ssn_address_count_7 ---> Done
ssn_address_count_14 ---> Done
ssn_address_count_30 ---> Done
Run time for entity ssn_address -------- 2.6421890258789062s

ssn_zip5_day_since ---> Done
ssn_zip5_count_0 ---> Done
ssn_zip5_count_1 ---> Done
ssn_zip5_count_3 ---> Done
ssn_zip5_count_7 ---> Done
ssn_zip5_count_14 ---> Done
ssn_zip5_count_30 ---> Done
Run time for entity ssn_zip5 -------- 2.593801498413086s

ssn_dob_day_since ---> Done
ssn_dob_count_0 ---> Done
ssn_dob_count_1 ---> Done
ssn_dob_count_3 ---> Done
ssn_dob_count_7 ---> Done
ssn_dob_count_14 ---> Done
ssn_dob_count_30 ---> Done
Run time for entity ssn_dob -------- 2.6196646690368652s

ssn_homephone_day_since ---> Done
ssn_homephone_count_0 ---> Done
ssn_homephone_count_1 ---> Done
ssn_homephone_count_3 ---> Done
ssn_homephone_count_7 ---> Done
ssn_homephone_count_14 ---> Done
ssn_homephone_count_30 ---> Done


In [36]:
start = time.time()
for att in attributes:
    for d in ['0', '1']:
        for dd in ['3', '7', '14', '30']:
            vars[att + '_count_' + d + '_by_' + dd] = vars[att + '_count_' + d]/ (vars[att + '_count_' + dd]/ float(dd))

print(f'Total run time: {time.time() -start}s')

Total run time: 11.979225397109985s


In [37]:
len(vars.columns)

680

In [38]:
att1 = ['ssn','fulladdress','name_dob','name_fulladdress','fulladdress_dob','dob_homephone',
        'ssn_lastname', 'ssn_zip5','ssn_name', 'ssn_fulladdress', 'ssn_name_dob','ssn_name_fulladdress']


In [39]:
begin = time.time()
for entity in att1:
    for field in att1:
        st = time.time()
        if field == entity:
            continue
        else:
            df_c = data1[[ 'record', 'date', entity]]
            df_d = data1[['check_record', 'check_date', entity, field]]
            temp = pd.merge(df_c, df_d, left_on=entity, right_on=entity)
            # number of unique for particular field
        for offset_t in [1, 3, 7, 14, 30, 60]:
            count_day_df = temp[(temp.check_date >= (temp.date - dt.timedelta (offset_t)))
                                & (temp.record >= temp.check_record)]
            col_name = f'{entity}unique_count_for{field}_{offset_t}'
            mapper = count_day_df.groupby(['record'])[field].nunique()
            vars[col_name] = vars.record.map (mapper)
                 
        print(f'Run time for entity {entity} in field {field}---{time.time() - st}s')
    print(f'Total run time: {(time.time() - begin) / 60}min')

Run time for entity ssn in field fulladdress---5.186958312988281s
Run time for entity ssn in field name_dob---4.938953399658203s
Run time for entity ssn in field name_fulladdress---5.190995693206787s
Run time for entity ssn in field fulladdress_dob---4.977910041809082s
Run time for entity ssn in field dob_homephone---4.792834281921387s
Run time for entity ssn in field ssn_lastname---4.606612920761108s
Run time for entity ssn in field ssn_zip5---4.500347137451172s
Run time for entity ssn in field ssn_name---4.8299713134765625s
Run time for entity ssn in field ssn_fulladdress---8.214155435562134s
Run time for entity ssn in field ssn_name_dob---5.081653594970703s
Run time for entity ssn in field ssn_name_fulladdress---5.234021425247192s
Total run time: 0.9592402259508769min
Run time for entity fulladdress in field ssn---4.470510482788086s
Run time for entity fulladdress in field name_dob---4.844587326049805s
Run time for entity fulladdress in field name_fulladdress---4.891624212265015s
Ru

Run time for entity ssn_fulladdress in field ssn_zip5---5.117923974990845s
Run time for entity ssn_fulladdress in field ssn_name---5.316695213317871s
Run time for entity ssn_fulladdress in field ssn_name_dob---5.2800469398498535s
Run time for entity ssn_fulladdress in field ssn_name_fulladdress---5.353001832962036s
Total run time: 10.090092984835307min
Run time for entity ssn_name_dob in field ssn---4.642644882202148s
Run time for entity ssn_name_dob in field fulladdress---5.056262731552124s
Run time for entity ssn_name_dob in field name_dob---5.079220533370972s
Run time for entity ssn_name_dob in field name_fulladdress---5.409193754196167s
Run time for entity ssn_name_dob in field fulladdress_dob---5.149897336959839s
Run time for entity ssn_name_dob in field dob_homephone---4.9993250370025635s
Run time for entity ssn_name_dob in field ssn_lastname---4.829537868499756s
Run time for entity ssn_name_dob in field ssn_zip5---4.706472158432007s
Run time for entity ssn_name_dob in field ssn_

In [40]:
vars

,record,date,ssn,firstname,lastname,address,zip5,dob,homephone,fraud_label,...,ssn_name_fulladdressunique_count_forssn_fulladdress_7,ssn_name_fulladdressunique_count_forssn_fulladdress_14,ssn_name_fulladdressunique_count_forssn_fulladdress_30,ssn_name_fulladdressunique_count_forssn_fulladdress_60,ssn_name_fulladdressunique_count_forssn_name_dob_1,ssn_name_fulladdressunique_count_forssn_name_dob_3,ssn_name_fulladdressunique_count_forssn_name_dob_7,ssn_name_fulladdressunique_count_forssn_name_dob_14,ssn_name_fulladdressunique_count_forssn_name_dob_30,ssn_name_fulladdressunique_count_forssn_name_dob_60
0,1,2016-01-01,379070012,XRRAMMTR,SMJETJMJ,6861 EUTST PL,02765,000000-1,1797504115,0,...,1,1,1,1,1,1,1,1,1,1
1,2,2016-01-01,387482503,MAMSTUJR,RTTEMRRR,7280 URASA PL,57169,19340615,4164239415,1,...,1,1,1,1,1,1,1,1,1,1
2,3,2016-01-01,200332444,SZMMUJEZS,EUSEZRAE,5581 RSREX LN,56721,000000-3,0216537580,0,...,1,1,1,1,1,1,1,1,1,1
3,4,2016-01-01,747451317,SJJZSXRSZ,ETJXTXXS,1387 UJZXJ RD,35286,19440430,0132144161,0,...,1,1,1,1,1,1,1,1,1,1
4,5,2016-01-01,024065868,SSSXUEJMS,SSUUJXUZ,279 EAASA WY,03173,19980315,6101082272,0,...,1,1,1,1,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999995,999996,2016-12-31,766104848,XUSAAURMZ,EJXXEXZX,8079 EJRUJ AVE,54890,19550418,8236558223,0,...,1,1,1,1,1,1,1,1,1,1
999996,999997,2016-12-31,445178058,SAAJJEMTX,UMAUETJX,4817 UMSXT AVE,74683,19150624,6790134432,0,...,1,1,1,1,1,1,1,1,1,1
999997,999998,2016-12-31,025283410,ESRXXRSUX,EUEJRJJE,7562 RJMAZ ST,86805,19880628,3997657622,0,...,1,1,1,1,1,1,1,1,1,1
999998,999999,2016-12-31,361232148,XXMZSRXMA,RSRSUJSS,3408 UZZAM WY,49183,20120302,5935293731,0,...,1,1,1,1,1,1,1,1,1,1
